<a href="https://colab.research.google.com/github/ValentinCord/HandOnAI_1/blob/main/Fire_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ **I**. Lecture des données depuis le cluster

✅ **II**. Fusion des données avec suppression des fichiers copiés

✅ **III**. Sauvegarde des données sur google.drive

✅ **IV**. Suppression des données abérrantes 

✅ **V**. Ajout de données utiles

✅ **VI**. Suppresion des doublons

✅ **VII**. Description des données

# **I. Reading database**

In [1]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Tue Nov  1 14:59:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
bases_path_after="bases"
if os.path.exists(bases_path_after) == False:
    os.makedirs(bases_path_after)

In [3]:
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_1.tar
!tar xf FIRE_DATABASE_1.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_1.tar

!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_2.tar
!tar xf FIRE_DATABASE_2.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_2.tar

!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
!tar xf FIRE_DATABASE_3.tar -C 'bases' --one-top-level
!rm FIRE_DATABASE_3.tar

--2022-11-01 14:59:06--  https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_1.tar
Resolving cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)... 193.190.209.220
Connecting to cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466772992 (445M) [application/octet-stream]
Saving to: ‘FIRE_DATABASE_1.tar’

FIRE_DATABASE_1.tar 100%[===================>] 445.15M  17.5MB/s    in 26s     

2022-11-01 14:59:33 (17.2 MB/s) - ‘FIRE_DATABASE_1.tar’ saved [466772992/466772992]

--2022-11-01 14:59:35--  https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_2.tar
Resolving cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)... 193.190.209.220
Connecting to cluster.ig.umons.ac.be (cluster.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1163223040 (1.1G) [application/octet-stream]
Saving to: ‘FIRE_DATABASE_2.tar’

FIRE_DATABASE_2.tar 100%[====

# **II. Merge data**

In [4]:
!rm -r db
!mkdir -p db/fire db/start_fire db/no_fire

rm: cannot remove 'db': No such file or directory


In [5]:
!find bases/FIRE_DATABASE_1/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/FIRE_DATABASE_2/fire ! -name '*copie*' -exec cp "{}" db/fire \;
!find bases/FIRE_DATABASE_3/fire ! -name '*copie*' -exec cp "{}" db/fire \;

!find bases/FIRE_DATABASE_1/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/FIRE_DATABASE_2/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;
!find bases/FIRE_DATABASE_3/start_fire ! -name '*copie*' -exec cp "{}" db/start_fire \;

!find bases/FIRE_DATABASE_1/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/FIRE_DATABASE_2/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;
!find bases/FIRE_DATABASE_3/no_fire ! -name '*copie*' -exec cp "{}" db/no_fire \;

cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_1/fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_2/fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_3/fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_1/start_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_2/start_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_3/start_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_1/no_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_2/no_fire'
cp: -r not specified; omitting directory 'bases/FIRE_DATABASE_3/no_fire'


# **III. Store database on google.drive**



In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
!cp -av '/content/db/' '/content/gdrive/MyDrive/HandOnAI_1_Fire/'

'/content/db/' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/'
'/content/db/fire' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire'
'/content/db/fire/00000167.jpg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/00000167.jpg'
'/content/db/fire/images (60).jpeg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/images (60).jpeg'
'/content/db/fire/téléchargement (51).jpeg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/téléchargement (51).jpeg'
'/content/db/fire/images - 2022-05-04T104008.293.jpeg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/images - 2022-05-04T104008.293.jpeg'
'/content/db/fire/00000016.jpg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/00000016.jpg'
'/content/db/fire/00000140.jpg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/00000140.jpg'
'/content/db/fire/00000209.jpg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/00000209.jpg'
'/content/db/fire/images - 2022-05-04T104011.706.jpeg' -> '/content/gdrive/MyDrive/HandOnAI_1_Fire/fire/images - 2022-05-04

In [8]:
drive_data_folder = '/content/gdrive/MyDrive/HandOnAI_1_Fire/'

# **IV. Delete wrong data**

In [9]:
img_aberrant = {
    'fire': ['00000335.jpg',
             'images - 2022-05-04T104033.252.jpeg',
             'images - 2022-05-04T104034.807.jpeg',
             'images - 2022-05-04T104044.688.jpeg',
             'images - 2022-05-04T104043.935.jpeg',
             'images - 2022-05-04T104047.349.jpeg'],
    'start_fire': [],
    'no_fire': []
    }

for classe, images in img_aberrant.items():
  
  for image in images:
    
    path = drive_data_folder + classe + '/' + image
    
    if os.path.exists(path):
      os.remove(path)

    else:
      print(f'{path} not found')


# **V. Adding data**

In [10]:
!pip install git+https://github.com/Joeclinton1/google-images-download.git

from google_images_download import google_images_download

# instantiate the class
response = google_images_download.googleimagesdownload()
arguments = {"keywords": "forest, red panda in a forest, sunset in a forest, fall in forest, fire forest",
             "limit": 50, "print_urls": False}
paths = response.download(arguments)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Joeclinton1/google-images-download.git to /tmp/pip-req-build-63vvdwh_
  Running command git clone -q https://github.com/Joeclinton1/google-images-download.git /tmp/pip-req-build-63vvdwh_
     |████████████████████████████████| 995 kB 7.4 MB/s 
     |████████████████████████████████| 140 kB 69.1 MB/s 
     |████████████████████████████████| 384 kB 68.1 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Created wheel for google-images-download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=16299 sha256=e391fbbb716bb293de94cdce213eb6af214d0e46562e77d9dee9554053754054
  Stored in directory: /tmp/pip-ephem-wheel-cache-qctyuik5/wheels/79/5c/d2/2569a4f55ff29b3c4d6b1114d5110a70a79afbf381e6eb3e87
Successfully built google-images-download
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urlli

# **VI. Delete duplicate data**

In [11]:
!pip install ImageHash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 296 kB 9.2 MB/s 


In [12]:
import imagehash
from tqdm import tqdm
from PIL import Image

In [13]:
from traitlets.traitlets import default
classes = ['fire/', 'start_fire/', 'no_fire/']

hash_dict = {}

for classe in classes:

  path = drive_data_folder + classe

  for image in tqdm(os.listdir(path)):

    path_image = path + image

    hash = imagehash.dhash(Image.open(path_image))

    if hash not in hash_dict:
      hash_dict[hash] = path_image

    else:
      #print(f'{hash_dict[hash]} is equal to {path_image}')
      os.remove(path_image)
      #print(f'{path_image} removed')


 68%|██████▊   | 527/775 [00:05<00:03, 69.16it/s]/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag)
100%|██████████| 500/500 [00:16<00:00, 30.73it/s]


# **VII. Data Description**


In [14]:
import os

path_fire = drive_data_folder + 'fire'
path_start_fire = drive_data_folder + 'start_fire'
path_no_fire = drive_data_folder + 'no_fire'

print(f'Fire : {len(os.listdir(path_fire))} images')
print(f'Start fire : {len(os.listdir(path_start_fire))} images')
print(f'No fire : {len(os.listdir(path_no_fire))} images')

Fire : 694 images
Start fire : 492 images
No fire : 497 images
